# Pipeline completo: Preprocesado, Modelado y Evaluación en Google Colab
Este notebook ejecuta el flujo completo para clasificación de tumores cerebrales en imágenes MRI: descarga/preprocesado, entrenamiento de un modelo CNN y evaluación.

---

**Flujo del notebook:**
1. Instalación de dependencias y configuración de entorno
2. Descarga y organización del dataset
3. Preprocesado y guardado de imágenes
4. Preparación de arrays de datos y etiquetas
5. División en conjuntos de entrenamiento, validación y test
6. Definición, entrenamiento y evaluación del modelo

Cada sección está documentada para facilitar la revisión y comprensión del proceso por parte del profesor.

**Asegúrate de tener tu archivo `kaggle.json` para descargar el dataset desde Kaggle.**

## 1. Instalación de dependencias y configuración de entorno Colab

In [ ]:
!pip install kaggle opencv-python-headless seaborn tensorflow scikit-learn --quiet
import os, shutil, zipfile, cv2, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import datetime
import tensorflow as tf

## 2. Descarga del dataset desde Kaggle
Se descarga el dataset de imágenes MRI desde Kaggle y se prepara la estructura de carpetas necesaria en el entorno de Google Colab.

## 2. Descarga del dataset desde Kaggle

In [ ]:
from google.colab import files
files.upload()
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 0o600)
!kaggle datasets download -d orvile/brain-cancer-mri-dataset -p /content/data --unzip

## 3. Reorganización y verificación de carpetas
Se reorganizan las carpetas de clases y se verifica la cantidad de imágenes disponibles por clase.

## 3. Reorganización y verificación de carpetas

In [ ]:
os.makedirs('/content/data/original', exist_ok=True)
clases = ['brain_glioma', 'brain_menin', 'brain_tumor']
for root, dirs, files in os.walk('/content/data'):
    for clase in clases:
        if clase in dirs:
            origen = os.path.join(root, clase)
            destino = f'/content/data/original/{clase}'
            if origen != destino and not os.path.exists(destino):
                shutil.move(origen, destino)
for dirpath, dirnames, filenames in os.walk('/content/data', topdown=False):
    if dirpath == '/content/data': continue
    if not dirnames and not filenames:
        os.rmdir(dirpath)
for clase in clases:
    n = len(glob(f'/content/data/original/{clase}/*.jpg'))
    print(f'{clase}: {n} imágenes')

## 4. Preprocesado de imágenes y guardado
Se aplica el preprocesamiento a las imágenes originales y se guardan las imágenes procesadas en carpetas separadas.

## 4. Preprocesado de imágenes y guardado

In [ ]:
def preprocesar_imagen(img_path, size=(128,128)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        try:
            with Image.open(img_path) as pil_img:
                img = np.array(pil_img.convert('L'))
        except Exception as e:
            print(f'[ERROR] No se pudo cargar la imagen: {img_path} ({e})')
            return np.zeros(size)
    _, thresh = cv2.threshold(img, 5, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(thresh)
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)
        img = img[y:y+h, x:x+w]
    img = cv2.resize(img, size)
    img = cv2.medianBlur(img, 5)
    img = cv2.equalizeHist(img)
    img = img.astype(np.float32)
    img = (img - img.min()) / (img.max() - img.min() + 1e-8)
    return img

os.makedirs('/content/data/preprocesadas', exist_ok=True)
for clase in clases:
    out_dir = f'/content/data/preprocesadas/{clase}'
    os.makedirs(out_dir, exist_ok=True)
    for img_path in glob(f'/content/data/original/{clase}/*.jpg'):
        img_proc = preprocesar_imagen(img_path)
        img_uint8 = (img_proc * 255).clip(0,255).astype(np.uint8)
        Image.fromarray(img_uint8).save(os.path.join(out_dir, os.path.basename(img_path)))

## 5. Preparación de arrays de datos y etiquetas
Se cargan las imágenes preprocesadas, se convierten en arrays NumPy y se generan las etiquetas correspondientes para su uso en el modelado.

## 5. Preparación de arrays de datos y etiquetas

In [ ]:
X, y = [], []
clase_a_idx = {c: i for i, c in enumerate(clases)}
n_imagenes = 2000
for clase in clases:
    imgs = glob(f'/content/data/preprocesadas/{clase}/*.jpg')[:n_imagenes]
    for img_path in imgs:
        img = preprocesar_imagen(img_path, size=(128,128))
        X.append(img)
        y.append(clase_a_idx[clase])
X = np.array(X)
y = np.array(y)
X = X.astype('float32') / 255.0
X = np.expand_dims(X, axis=-1)
print('Shape X:', X.shape)
print('Shape y:', y.shape)

## 6. División en train, val y test

In [ ]:
y_cat = to_categorical(y, num_classes=len(clases))
# División: 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y_cat, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=np.argmax(y_temp, axis=1), random_state=42)
print(f'Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}')

## 7. Modelado: Definición y entrenamiento de una CNN simple

In [ ]:
# Arquitectura óptima basada en la versión local
def crear_modelo(conv_filters, dense_units, dropout_conv, dropout_dense, lr):
    model = Sequential()
    model.add(Conv2D(conv_filters[0], (3,3), activation='relu', input_shape=X_train.shape[1:]))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(dropout_conv))
    model.add(Conv2D(conv_filters[1], (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(dropout_conv))
    model.add(Conv2D(conv_filters[2], (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(dropout_conv + 0.1))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_dense))
    model.add(Dense(len(clases), activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model
# Hiperparámetros óptimos
best_params = {
    'conv_filters': [16, 32, 64],
    'dense_units': 64,
    'dropout_conv': 0.1,
    'dropout_dense': 0.1,
    'lr': 0.0005
}
model = crear_modelo(**best_params)
model.summary()

## 8. Entrenamiento del modelo

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=40,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)
# Visualización de la historia de entrenamiento
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.tight_layout()
plt.show()

## 9. Evaluación y visualización de resultados

In [ ]:
# Evaluación en test
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en test: {test_acc:.4f}')
# Matriz de confusión
y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(model.predict(X_test), axis=1)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=clases, yticklabels=clases)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de confusión')
plt.show()
# Reporte de clasificación
print(classification_report(y_true, y_pred, target_names=clases))